In [1]:
import os
import cv2
import numpy as np
import random
import shutil

# 定义Potsdam数据集的标签像素值与类别对应关系
potsdam_classes = {
    1: {'name': 'Building', 'color': (255, 0, 0)},
    2: {'name': 'Impervious surfaces', 'color': (0, 0, 255)},
    3: {'name': 'Car', 'color': (0, 255, 255)},
    4: {'name': 'Tree', 'color': (0, 255, 0)},
    5: {'name': 'Low vegetation', 'color': (255, 255, 0)},
    6: {'name': 'Clutter/background', 'color': (255, 255, 255)}
}

# 获取类别名称列表，用于后续的可视化
class_names = [info['name'] for info in potsdam_classes.values()]

def convert_potsdam_to_yolo_seg(labels_dir, images_dir, output_images_dir, output_labels_dir):
    # 创建输出目录
    os.makedirs(output_images_dir, exist_ok=True)
    os.makedirs(output_labels_dir, exist_ok=True)

    # 遍历图片和分割标签
    image_files = os.listdir(images_dir)
    for img_file in image_files:
        img_name = os.path.splitext(img_file)[0]
        img_path = os.path.join(images_dir, img_file)
        label_path = os.path.join(labels_dir, img_name.replace('IRRG', 'label') + '.png')

        # 检查图片文件是否存在
        if not os.path.exists(img_path):
            print(f"图片文件不存在: {img_path}")
            continue

        # 检查标签文件是否存在
        if not os.path.exists(label_path):
            print(f"标签文件不存在: {label_path}")
            continue

        # 读取图片获取尺寸信息
        img = cv2.imread(img_path)
        if img is None:
            print(f"无法读取图片: {img_path}")
            continue
        height, width = img.shape[:2]

        # 复制图片到输出目录
        shutil.copy(img_path, os.path.join(output_images_dir, img_file))

        # 读取分割标签
        label = cv2.imread(label_path, cv2.IMREAD_COLOR)
        if label is None:
            print(f"无法读取标签: {label_path}")
            continue

        # 将颜色标签转换为类别ID掩码
        id_mask = np.zeros((height, width), dtype=np.uint8)
        for cls_id, color_info in potsdam_classes.items():
            color = color_info['color']
            mask = np.all(label == color, axis=-1)
            id_mask[mask] = cls_id

        # 找到每个类别的轮廓
        for cls_id in np.unique(id_mask):
            if cls_id == 0:  # 跳过背景
                continue

            # 获取当前类别的掩码
            mask = (id_mask == cls_id).astype(np.uint8)

            # 寻找轮廓
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

            for contour in contours:
                # 过滤掉太小的轮廓
                if cv2.contourArea(contour) < 10:
                    continue

                # 获取分割点信息，按YOLO-seg格式要求的格式
                segmentation = contour.flatten().astype(np.float32)
                # 归一化坐标点到0-1范围
                segmentation[0::2] /= width
                segmentation[1::2] /= height

                # 写入txt文件
                output_path = os.path.join(output_labels_dir, img_name + '.txt')
                with open(output_path, 'a') as f:
                    line = f"{cls_id - 1} " + ' '.join(map(str, segmentation))
                    f.write(line + '\n')

def visualize_yolo_seg_labels(images_dir, labels_dir, output_dir, sample_size=5):
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)

    # 获取所有图片文件
    image_files = os.listdir(images_dir)
    total_images = len(image_files)
    if total_images == 0:
        print("未找到图片文件")
        return

    # 随机抽样
    sampled_indices = random.sample(range(total_images), min(sample_size, total_images))
    sampled_image_files = [image_files[i] for i in sampled_indices]

    # 遍历抽样的图片
    for img_file in sampled_image_files:
        img_name = os.path.splitext(img_file)[0]
        img_path = os.path.join(images_dir, img_file)
        label_path = os.path.join(labels_dir, img_name + '.txt')
        output_path = os.path.join(output_dir, img_name + '_visualized.jpg')

        # 检查图片文件是否存在
        if not os.path.exists(img_path):
            print(f"图片文件不存在: {img_path}")
            continue

        # 检查标签文件是否存在
        if not os.path.exists(label_path):
            print(f"标签文件不存在: {label_path}")
            continue

        # 读取图片
        img = cv2.imread(img_path)
        if img is None:
            print(f"无法读取图片: {img_path}")
            continue

        height, width = img.shape[:2]

        # 读取并解析txt文件
        with open(label_path, 'r') as f:
            lines = f.readlines()

        for line in lines:
            parts = line.strip().split()
            if len(parts) < 1:
                continue

            cls_id = int(parts[0])
            if cls_id < 0 or cls_id >= len(class_names):
                print(f"无效的类别ID: {cls_id} in {label_path}")
                continue

            # 解析分割点信息
            points = list(map(float, parts[1:]))
            if len(points) % 2 != 0:
                print(f"无效的分割点信息 in {label_path}")
                continue

            # 归一化坐标转换为原始图片坐标
            points = np.array(points, dtype=np.float32)
            points[0::2] *= width
            points[1::2] *= height
            points = points.astype(np.int32)

            # 绘制分割区域
            pts = points.reshape((-1, 2))
            # 使用与映射表一致的颜色
            color = potsdam_classes[cls_id+1]['color']  # 注意：cls_id从0开始，而potsdam_classes从1开始
            cv2.polylines(img, [pts], isClosed=True, color=color, thickness=2)
            cv2.putText(img, class_names[cls_id], (int(pts[0][0]), int(pts[0][1]) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 3)

        # 保存可视化结果
        cv2.imwrite(output_path, img)
        print(f"已保存可视化结果: {output_path}")


labels_dir = '/Users/makotoxu/Documents/postdam_cropped/train/label'  # 分割标签所在目录
images_dir = '/Users/makotoxu/Documents/postdam_cropped/train/data'  # 原始图片所在目录
output_labels_dir = '/Users/makotoxu/Documents/postdam_cropped/labels/train'  # 输出的YOLO-seg格式txt文件所在目录
output_images_dir = '/Users/makotoxu/Documents/postdam_cropped/images/train'  # 输出的图片文件所在目录
visualization_dir = '/Users/makotoxu/Documents/postdam_cropped/vis/train'   # 可视化结果保存目录
# 转换Potsdam数据集的分割标签为YOLO-seg格式，并复制图片到输出目录
convert_potsdam_to_yolo_seg(labels_dir, images_dir, output_images_dir, output_labels_dir)

# 对生成的txt文件进行验证
visualize_yolo_seg_labels(output_images_dir, output_labels_dir, visualization_dir, sample_size=20)

已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/train/block_top_potsdam_5_14_IRRG_16_1_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/train/block_top_potsdam_3_11_IRRG_15_6_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/train/block_top_potsdam_4_13_IRRG_11_7_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/train/block_top_potsdam_2_14_IRRG_7_23_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/train/block_top_potsdam_4_15_IRRG_18_18_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/train/block_top_potsdam_3_12_IRRG_1_23_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/train/block_top_potsdam_7_13_IRRG_16_3_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/train/block_top_potsdam_4_13_IRRG_11_12_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/train/block_top_potsdam_5_12_IRRG_22_0_visualized.jpg
已保存可视化结果

In [2]:
import os
import cv2
import numpy as np
import random
import shutil

# 定义Potsdam数据集的标签像素值与类别对应关系
potsdam_classes = {
    1: {'name': 'Building', 'color': (255, 0, 0)},
    2: {'name': 'Impervious surfaces', 'color': (0, 0, 255)},
    3: {'name': 'Car', 'color': (0, 255, 255)},
    4: {'name': 'Tree', 'color': (0, 255, 0)},
    5: {'name': 'Low vegetation', 'color': (255, 255, 0)},
    6: {'name': 'Clutter/background', 'color': (255, 255, 255)}
}

# 获取类别名称列表，用于后续的可视化
class_names = [info['name'] for info in potsdam_classes.values()]

def convert_potsdam_to_yolo_seg(labels_dir, images_dir, output_images_dir, output_labels_dir):
    # 创建输出目录
    os.makedirs(output_images_dir, exist_ok=True)
    os.makedirs(output_labels_dir, exist_ok=True)

    # 遍历图片和分割标签
    image_files = os.listdir(images_dir)
    for img_file in image_files:
        img_name = os.path.splitext(img_file)[0]
        img_path = os.path.join(images_dir, img_file)
        label_path = os.path.join(labels_dir, img_name.replace('IRRG', 'label') + '.png')

        # 检查图片文件是否存在
        if not os.path.exists(img_path):
            print(f"图片文件不存在: {img_path}")
            continue

        # 检查标签文件是否存在
        if not os.path.exists(label_path):
            print(f"标签文件不存在: {label_path}")
            continue

        # 读取图片获取尺寸信息
        img = cv2.imread(img_path)
        if img is None:
            print(f"无法读取图片: {img_path}")
            continue
        height, width = img.shape[:2]

        # 复制图片到输出目录
        shutil.copy(img_path, os.path.join(output_images_dir, img_file))

        # 读取分割标签
        label = cv2.imread(label_path, cv2.IMREAD_COLOR)
        if label is None:
            print(f"无法读取标签: {label_path}")
            continue

        # 将颜色标签转换为类别ID掩码
        id_mask = np.zeros((height, width), dtype=np.uint8)
        for cls_id, color_info in potsdam_classes.items():
            color = color_info['color']
            mask = np.all(label == color, axis=-1)
            id_mask[mask] = cls_id

        # 找到每个类别的轮廓
        for cls_id in np.unique(id_mask):
            if cls_id == 0:  # 跳过背景
                continue

            # 获取当前类别的掩码
            mask = (id_mask == cls_id).astype(np.uint8)

            # 寻找轮廓
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

            for contour in contours:
                # 过滤掉太小的轮廓
                if cv2.contourArea(contour) < 10:
                    continue

                # 获取分割点信息，按YOLO-seg格式要求的格式
                segmentation = contour.flatten().astype(np.float32)
                # 归一化坐标点到0-1范围
                segmentation[0::2] /= width
                segmentation[1::2] /= height

                # 写入txt文件
                output_path = os.path.join(output_labels_dir, img_name + '.txt')
                with open(output_path, 'a') as f:
                    line = f"{cls_id - 1} " + ' '.join(map(str, segmentation))
                    f.write(line + '\n')

def visualize_yolo_seg_labels(images_dir, labels_dir, output_dir, sample_size=5):
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)

    # 获取所有图片文件
    image_files = os.listdir(images_dir)
    total_images = len(image_files)
    if total_images == 0:
        print("未找到图片文件")
        return

    # 随机抽样
    sampled_indices = random.sample(range(total_images), min(sample_size, total_images))
    sampled_image_files = [image_files[i] for i in sampled_indices]

    # 遍历抽样的图片
    for img_file in sampled_image_files:
        img_name = os.path.splitext(img_file)[0]
        img_path = os.path.join(images_dir, img_file)
        label_path = os.path.join(labels_dir, img_name + '.txt')
        output_path = os.path.join(output_dir, img_name + '_visualized.jpg')

        # 检查图片文件是否存在
        if not os.path.exists(img_path):
            print(f"图片文件不存在: {img_path}")
            continue

        # 检查标签文件是否存在
        if not os.path.exists(label_path):
            print(f"标签文件不存在: {label_path}")
            continue

        # 读取图片
        img = cv2.imread(img_path)
        if img is None:
            print(f"无法读取图片: {img_path}")
            continue

        height, width = img.shape[:2]

        # 读取并解析txt文件
        with open(label_path, 'r') as f:
            lines = f.readlines()

        for line in lines:
            parts = line.strip().split()
            if len(parts) < 1:
                continue

            cls_id = int(parts[0])
            if cls_id < 0 or cls_id >= len(class_names):
                print(f"无效的类别ID: {cls_id} in {label_path}")
                continue

            # 解析分割点信息
            points = list(map(float, parts[1:]))
            if len(points) % 2 != 0:
                print(f"无效的分割点信息 in {label_path}")
                continue

            # 归一化坐标转换为原始图片坐标
            points = np.array(points, dtype=np.float32)
            points[0::2] *= width
            points[1::2] *= height
            points = points.astype(np.int32)

            # 绘制分割区域
            pts = points.reshape((-1, 2))
            # 使用与映射表一致的颜色
            color = potsdam_classes[cls_id+1]['color']  # 注意：cls_id从0开始，而potsdam_classes从1开始
            cv2.polylines(img, [pts], isClosed=True, color=color, thickness=2)
            cv2.putText(img, class_names[cls_id], (int(pts[0][0]), int(pts[0][1]) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 3)

        # 保存可视化结果
        cv2.imwrite(output_path, img)
        print(f"已保存可视化结果: {output_path}")


labels_dir = '/Users/makotoxu/Documents/postdam_cropped/test/label'  # 分割标签所在目录
images_dir = '/Users/makotoxu/Documents/postdam_cropped/test/data'  # 原始图片所在目录
output_labels_dir = '/Users/makotoxu/Documents/postdam_cropped/labels/test'  # 输出的YOLO-seg格式txt文件所在目录
output_images_dir = '/Users/makotoxu/Documents/postdam_cropped/images/test'  # 输出的图片文件所在目录
visualization_dir = '/Users/makotoxu/Documents/postdam_cropped/vis/test'   # 可视化结果保存目录
# 转换Potsdam数据集的分割标签为YOLO-seg格式，并复制图片到输出目录
convert_potsdam_to_yolo_seg(labels_dir, images_dir, output_images_dir, output_labels_dir)

# 对生成的txt文件进行验证
visualize_yolo_seg_labels(output_images_dir, output_labels_dir, visualization_dir, sample_size=20)

已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/test/block_top_potsdam_7_9_IRRG_20_11_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/test/block_top_potsdam_7_11_IRRG_23_25_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/test/block_top_potsdam_7_7_IRRG_25_4_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/test/block_top_potsdam_7_7_IRRG_3_3_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/test/block_top_potsdam_7_9_IRRG_3_3_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/test/block_top_potsdam_7_12_IRRG_12_11_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/test/block_top_potsdam_7_11_IRRG_14_17_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/test/block_top_potsdam_7_8_IRRG_15_12_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/test/block_top_potsdam_7_12_IRRG_13_23_visualized.jpg
已保存可视化结果: /Users/mak

In [3]:
labels_dir = '/Users/makotoxu/Documents/postdam_cropped/val/label'  # 分割标签所在目录
images_dir = '/Users/makotoxu/Documents/postdam_cropped/val/data'  # 原始图片所在目录
output_labels_dir = '/Users/makotoxu/Documents/postdam_cropped/labels/val'  # 输出的YOLO-seg格式txt文件所在目录
output_images_dir = '/Users/makotoxu/Documents/postdam_cropped/images/val'  # 输出的图片文件所在目录
visualization_dir = '/Users/makotoxu/Documents/postdam_cropped/vis/val'   # 可视化结果保存目录

# 对生成的txt文件进行验证
visualize_yolo_seg_labels(output_images_dir, output_labels_dir, visualization_dir, sample_size=20)

已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/val/block_top_potsdam_6_9_IRRG_1_10_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/val/block_top_potsdam_6_13_IRRG_9_12_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/val/block_top_potsdam_6_9_IRRG_24_2_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/val/block_top_potsdam_6_11_IRRG_16_16_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/val/block_top_potsdam_6_12_IRRG_13_25_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/val/block_top_potsdam_6_14_IRRG_6_16_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/val/block_top_potsdam_6_11_IRRG_7_20_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/val/block_top_potsdam_6_11_IRRG_10_22_visualized.jpg
已保存可视化结果: /Users/makotoxu/Documents/postdam_cropped/vis/val/block_top_potsdam_6_9_IRRG_23_8_visualized.jpg
已保存可视化结果: /Users/makotoxu/Do

In [ ]:
import cv2
import numpy as np

# 定义Potsdam数据集的标签像素值与类别对应关系
potsdam_classes = {
    1: {'name': 'Building', 'color': (255, 0, 0)},
    2: {'name': 'Impervious surfaces', 'color': (0, 0, 255)},
    3: {'name': 'Car', 'color': (0, 255, 255)},
    4: {'name': 'Tree', 'color': (0, 255, 0)},
    5: {'name': 'Low vegetation', 'color': (255, 255, 0)},
    6: {'name': 'Clutter/background', 'color': (255, 255, 255)}
}

# 获取类别名称列表，用于后续的可视化
class_names = [info['name'] for info in potsdam_classes.values()]

# 创建一个空白图像用于绘制颜色条
width = 300
height = len(potsdam_classes) * 50  # 每个类别占用 50 像素高度
image = np.ones((height, width, 3), dtype=np.uint8) * 255

# 绘制每个类别的颜色条
for idx, (cls_id, class_info) in enumerate(potsdam_classes.items()):
    color = class_info['color']
    name = class_info['name']
    
    # 填充颜色条区域
    cv2.rectangle(image, (0, idx * 50), (width, (idx + 1) * 50), color, thickness=-1)
    
    # 添加类别名称
    cv2.putText(image, name, (10, (idx + 1) * 50 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

# 显示和保存图像
cv2.imshow("Potsdam Color Map", image)
cv2.imwrite("potsdam_color_map.png", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

2025-03-13 21:59:46.775 python[86588:25639377] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-13 21:59:46.776 python[86588:25639377] +[IMKInputSession subclass]: chose IMKInputSession_Modern
